In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# To view the free memory and used memory for GPU
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
gpu = GPUs[0]

def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print('GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB'.format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.9 GB  I Proc size: 150.1 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


In [0]:
import os
os.chdir("/content/gdrive/My Drive/flair/dataset")  

In [4]:
! pip install flair

     |████████████████████████████████| 143kB 3.5MB/s 
     |████████████████████████████████| 798kB 50.5MB/s 
     |████████████████████████████████| 655kB 49.0MB/s 
     |████████████████████████████████| 133kB 52.1MB/s 
     |████████████████████████████████| 1.0MB 45.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Stored in directory: /root/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
  Stored in directory: /root/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
  Stored in directory: /root/.cache/pip/wheels/15/ee/a8/6112173f1386d33eebedb3f73429cfa41a4c3084556bcee254
Successfully built mpld3 regex sqlitedict segtok


In [5]:
# imports 
from flair.datasets import Corpus
from flair.data_fetcher import NLPTaskDataFetcher
from typing import List

# columns of "gold standard" ner annotations and text
columns = {1: 'text', 3: 'ner'}

# folder where training and test data are
data_folder = '/content/gdrive/My Drive/flair/dataset/'

# 2. what tag do we want to predict?
tag_type = 'ner'

# 1.0 is full data, try a much smaller number like 0.1 to test run the code
downsample = 1.0 

## your train file name
train_file = 'train_res_bilou.txt'

## your test file name
test_file = 'test_res_bilou.txt'

# 1. get the corpus
corpus: Corpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns,
                                                             train_file=train_file,
                                                             test_file=test_file,
                                                           dev_file=None).downsample(downsample)
print(corpus)

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type='ner')
print(tag_dictionary.idx2item)

2019-06-18 18:43:21,986 Reading data from /content/gdrive/My Drive/flair/dataset
2019-06-18 18:43:21,987 Train: /content/gdrive/My Drive/flair/dataset/train_res_bilou.txt
2019-06-18 18:43:21,991 Dev: None
2019-06-18 18:43:21,993 Test: /content/gdrive/My Drive/flair/dataset/test_res_bilou.txt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated function (or staticmethod) load_column_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:312: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  train_file, column_format
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:318: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  test_file, column_format


Corpus: 301 train + 33 dev + 143 test sentences
[b'<unk>', b'O', b'', b'at"', b'worked', b'<START>', b'<STOP>']


In [6]:
# 4. initialize embeddings. Experiment with different embedding types to see what gets the best results
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings,FlairEmbeddings

embedding_types: List[TokenEmbeddings] = [
    WordEmbeddings('glove')
    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings (needs a LONG time to train :-)
    #FlairEmbeddings('news-forward'),
    #FlairEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type='ner',
                                        use_crf=True)


2019-06-18 18:43:43,868 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmp0u2dg9ge


100%|██████████| 160000128/160000128 [00:08<00:00, 18676097.34B/s]

2019-06-18 18:43:53,014 copying /tmp/tmp0u2dg9ge to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2019-06-18 18:43:53,300 removing temp file /tmp/tmp0u2dg9ge
2019-06-18 18:43:53,781 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim not found in cache, downloading to /tmp/tmp85yxy3p2


100%|██████████| 21494764/21494764 [00:01<00:00, 12698077.35B/s]

2019-06-18 18:43:56,010 copying /tmp/tmp85yxy3p2 to cache at /root/.flair/embeddings/glove.gensim
2019-06-18 18:43:56,038 removing temp file /tmp/tmp85yxy3p2



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

## give your model a name and folder of your choice. Your model will be saved there for loading later 
## you can run this notebook many times with different embeddings/params and save the models with different names
model_name = 'resources/taggers/resume-ner-4'

# 7. start training - you can experiment with batch size if you get memory errors
# how many epochs does it take before the model stops showing improvement? Start with a big number like 150, and stop the code cell
# from running at any time - the framework will persist the best model even if you interrupt training. 
trainer.train(model_name,
              learning_rate=0.1,
              mini_batch_size=10,
              #anneal_with_restarts=True,
              max_epochs=150)

# 8. plot training curves (optional)
#from flair.visual.training_curves import Plotter
#plotter = Plotter()
#plotter.plot_training_curves('resources/taggers/example-ner/loss.tsv')
#plotter.plot_weights('resources/taggers/example-ner/weights.txt')

2019-06-17 21:03:45,290 ----------------------------------------------------------------------------------------------------
2019-06-17 21:03:45,292 Evaluation method: MICRO_F1_SCORE
2019-06-17 21:03:45,656 ----------------------------------------------------------------------------------------------------
2019-06-17 21:03:48,728 epoch 1 - iter 0/15 - loss 558.49407959
2019-06-17 21:03:52,543 epoch 1 - iter 1/15 - loss 388.78331757
2019-06-17 21:03:57,528 epoch 1 - iter 2/15 - loss 321.16821798
2019-06-17 21:04:06,113 epoch 1 - iter 3/15 - loss 282.82592773
2019-06-17 21:04:09,991 epoch 1 - iter 4/15 - loss 249.90119629
2019-06-17 21:04:12,162 epoch 1 - iter 5/15 - loss 222.93744151
2019-06-17 21:04:14,065 epoch 1 - iter 6/15 - loss 201.74871063
2019-06-17 21:04:16,091 epoch 1 - iter 7/15 - loss 185.63141251
2019-06-17 21:04:17,914 epoch 1 - iter 8/15 - loss 197.30933974
2019-06-17 21:04:20,061 epoch 1 - iter 9/15 - loss 182.12794876
2019-06-17 21:04:22,063 epoch 1 - iter 10/15 - loss 

{'dev_loss_history': [tensor(40.2810, device='cuda:0'),
  tensor(30.4474, device='cuda:0'),
  tensor(18.4309, device='cuda:0'),
  tensor(42.0059, device='cuda:0'),
  tensor(13.7058, device='cuda:0'),
  tensor(17.8702, device='cuda:0'),
  tensor(14.5216, device='cuda:0'),
  tensor(14.1801, device='cuda:0'),
  tensor(11.3878, device='cuda:0'),
  tensor(10.5237, device='cuda:0'),
  tensor(10.2383, device='cuda:0'),
  tensor(12.1768, device='cuda:0'),
  tensor(11.1960, device='cuda:0'),
  tensor(14.4164, device='cuda:0'),
  tensor(11.2008, device='cuda:0'),
  tensor(9.5299, device='cuda:0'),
  tensor(9.3600, device='cuda:0'),
  tensor(9.5517, device='cuda:0'),
  tensor(15.6692, device='cuda:0'),
  tensor(10.4840, device='cuda:0'),
  tensor(13.2713, device='cuda:0'),
  tensor(10.3083, device='cuda:0'),
  tensor(10.6555, device='cuda:0'),
  tensor(11.5229, device='cuda:0'),
  tensor(9.9069, device='cuda:0'),
  tensor(9.4518, device='cuda:0'),
  tensor(8.9999, device='cuda:0'),
  tensor(9.492

In [0]:
# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

## give your model a name and folder of your choice. Your model will be saved there for loading later 
## you can run this notebook many times with different embeddings/params and save the models with different names
model_name = 'resources/taggers/resume-ner-5'

# 7. start training - you can experiment with batch size if you get memory errors
# how many epochs does it take before the model stops showing improvement? Start with a big number like 150, and stop the code cell
# from running at any time - the framework will persist the best model even if you interrupt training. 
trainer.train(model_name,
              learning_rate=0.1,
              mini_batch_size=5,
              #anneal_with_restarts=True,
              max_epochs=100)

# 8. plot training curves (optional)
#from flair.visual.training_curves import Plotter
#plotter = Plotter()
#plotter.plot_training_curves('resources/taggers/example-ner/loss.tsv')
#plotter.plot_weights('resources/taggers/example-ner/weights.txt')

2019-06-18 20:22:34,617 epoch 33 - iter 18/61 - loss 19.79700766
2019-06-18 20:22:48,117 epoch 33 - iter 24/61 - loss 19.97154720
2019-06-18 20:23:03,797 epoch 33 - iter 30/61 - loss 20.73947377
2019-06-18 20:23:14,863 epoch 33 - iter 36/61 - loss 20.28685196
2019-06-18 20:23:24,542 epoch 33 - iter 42/61 - loss 19.67532731
2019-06-18 20:23:37,018 epoch 33 - iter 48/61 - loss 19.16641188
2019-06-18 20:23:53,244 epoch 33 - iter 54/61 - loss 19.22493715
2019-06-18 20:24:04,506 epoch 33 - iter 60/61 - loss 19.43153038
2019-06-18 20:24:05,029 ----------------------------------------------------------------------------------------------------
2019-06-18 20:24:05,031 EPOCH 33 done: loss 19.4315 - lr 0.0008 - bad epochs 0
2019-06-18 20:24:19,111 DEV : loss 16.214584350585938 - score 0.4997
2019-06-18 20:24:55,191 TEST : loss 17.176359176635742 - score 0.4641
2019-06-18 20:24:55,202 ----------------------------------------------------------------------------------------------------
2019-06-18 2